# **Entrainement - Modèle Inception**

## *Connexion Google Drive*

In [1]:
# Connexion au google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## *Installation des packages*

In [2]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 4.5 MB/s 
     |████████████████████████████████| 1.6 MB 37.4 MB/s 


In [8]:
pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## **Importation des librairies**





In [9]:
# Import Libraries
import numpy as np 
import pandas as pd 
import time
import itertools
from matplotlib import image

# Téléchargement de fichiers à partir de Collab
from google.colab import files

# file system libraries
import os
import os.path
from   os import path
import shutil

# confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# splits train folders into train/validation with stratification
import splitfolders  

# Images, Plotting
from skimage import io
import matplotlib.pyplot as plt
import tensorflow.keras.preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# tensorflow - CNNs
import tensorflow as tf
import keras_tuner as kt
import keras.backend as K
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import backend, models, layers, Sequential
from tensorflow.keras.layers import Input, Concatenate, Dense, Dropout, Flatten, Add
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.applications import DenseNet121,InceptionV3, Xception, ResNet101
from keras_tuner.tuners import Hyperband

## **Définition de la fonction F1-score (nécessaire pour le chargement du modèle**


In [10]:
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [11]:
def accuracy (y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    true_negatives = K.sum(K.round(K.clip((1 - y_true)*(1 - y_pred), 0, 1)))
    false_positives = K.sum(K.round(K.clip((1 - y_true)*(y_pred), 0, 1)))
    false_negatives = K.sum(K.round(K.clip((y_true)*(1 - y_pred), 0, 1)))
    acc_val = ((true_positives + true_negatives)/(true_positives + true_negatives + false_positives + false_negatives))
    return acc_val

In [12]:
# Constants
# Constants
FOLDERS     = ['Alzheimer_s Dataset','patients']
DIR_INPUT   = '/content/drive/MyDrive/Pathologia/Alzheimer'
DIR_WORK    = './'
DIR_MODELS  = os.path.join('/content/drive/MyDrive/Pathologia/Modèles/','models')
DIR_PATIENTS   = os.path.join('/content/drive/MyDrive/Pathologia/Alzheimer/Alzheimer_s Dataset/Recherche_alzheimer_patients')
CLASS_LIST  = ['Recherche', 'Archive']

# Set seeds for reproducibility 
SEED        = 1985
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [13]:
IMG_SIZE = [176,208]
BATCH_SIZE = 32

# Scale Images
patients_images  = ImageDataGenerator(rescale  = 1./255)

# Image Generators
patients_gen = patients_images.flow_from_directory(
    DIR_PATIENTS,
    target_size = IMG_SIZE,
    batch_size  = BATCH_SIZE,
    class_mode  = 'categorical',
    color_mode  = 'rgb',
    seed        = SEED,
    shuffle     = False
)

Found 1 images belonging to 1 classes.


# **Chargement du modèle Keras appris**

In [14]:
chargement_modele = load_model('/content/drive/MyDrive/Pathologia/Modèles/models/DenseNet121_model_added_capacity.keras',custom_objects={"f1_score": f1_score, "accuracy" : accuracy})

#**Prédiction de la maladie d'Alzheimer pour une IRM**


In [15]:
pred = chargement_modele.predict(patients_gen)
print(pred)

1/1 [==============================] - 9s 9s/step
[[5.5536310e-13 1.0000000e+00 4.0502053e-11]]


In [16]:
print("Probabilité d'avoir la maladie : ",pred[0,0])
print("Probabilité d'avoir une suspicion de la maladie : ",pred[0,2])
print("Probabilité de ne pas avoir la maladie : ",pred[0,1])

Probabilité d'avoir la maladie :  5.553631e-13
Probabilité d'avoir une suspicion de la maladie :  4.0502053e-11
Probabilité de ne pas avoir la maladie :  1.0


In [48]:
import sqlite3

In [49]:
conn = sqlite3.connect('/content/drive/MyDrive/db.sqlite3', timeout=10)

In [45]:
conn.execute("SELECT DATE()")

In [50]:
conn.execute("INSERT INTO webapp_result (Id_Patient, Prediction, Type) VALUES ('DP1',"+str(pred[0,1])+", 'DP')")

In [51]:
conn.commit()
conn.close()